# ACE-Step 1.5 🚀 (Google Colab & VS Code)

이 노트북은 **ACE-Step 1.5**를 Google Colab 또는 VS Code의 Colab 커널에서 실행하기 위한 통합 가이드입니다.

## 💻 1. VS Code에서 실행하는 방법 (권장)

현재 **Mac**에서 이 파일을 보고 계신다면, 반드시 커널을 Colab으로 연결해야 합니다:
1. **확장 프로그램 설치**: VS Code `확장(Extensions)` 뷰에서 **'Google Colab'**을 검색하여 설치합니다.
2. **커널 선택**: 우측 상단의 `커널 선택(Select Kernel)` 버튼을 클릭하고 **'Google Colab'**을 선택한 뒤 Google 계정으로 로그인합니다.
3. **런타임 선택**: 고성능 GPU(T4, L4 등)가 포함된 런타임을 선택하세요.

---

## 🛠️ 2. 환경 설정

### ⚠️ GPU 상태 확인

In [ ]:
import subprocess
import sys

def check_gpu():
    print("🔍 시스템 환경을 확인 중입니다...")
    
    try:
        # nvidia-smi 명령어 실행 확인
        result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.returncode == 0:
            print("✅ NVIDIA GPU가 감지되었습니다.")
            print(result.stdout.split('\n')[8]) # GPU 모델명 간략 출력
        else:
            print("❌ GPU를 찾을 수 없습니다. (NVIDIA 드라이버 응답 없음)")
            print("💡 Colab 메뉴 [런타임] -> [런타임 유형 변경]에서 'T4 GPU' 이상을 선택했는지 확인하세요.")
    except FileNotFoundError:
        print("❌ 'nvidia-smi' 명령어를 실행할 수 없습니다.")
        if sys.platform == 'darwin':
            print("💡 현재 'Local' Mac 커널이 연결된 것 같습니다. 우측 상단에서 'Google Colab' 커널로 변경해 주세요.")
        else:
            print("💡 GPU 가속이 없는 환경입니다. 모델 실행이 매우 느릴 수 있습니다.")

check_gpu()

### 📥 프로젝트 내려받기 및 의존성 설치

In [ ]:
import os
if not os.path.exists('/content/ACE-Step-1.5'):
    print("📂 리포지토리를 클론합니다...")
    !git clone https://github.com/ACE-Step/ACE-Step-1.5.git /content/ACE-Step-1.5

%cd /content/ACE-Step-1.5

# uv 설치 및 의존성 동기화
if os.system("command -v uv > /dev/null") != 0:
    print("📦 uv 패키지 매니저를 설치합니다...")
    !curl -LsSf https://astral.sh/uv/install.sh | sh
os.environ['PATH'] = f"{os.environ['HOME']}/.local/bin:{os.environ['PATH']}"

print("⚙️ 라이브러리를 동기화합니다...")
!uv sync

print("🚀 nano-vllm 및 추가 의존성 설치...")
# 가상 환경 내부에 직접 설치를 보장하기 위해 uv pip install 사용
!uv pip install setuptools wheel
!uv pip install -e acestep/third_parts/nano-vllm

# 설치 확인 테스트
try:
    import nano_vllm
    print("✅ nano-vllm 설치 확인 완료!")
except ImportError:
    # uv run 환경을 위해 PYTHONPATH에 추가 시도
    import sys
    sys.path.append(os.path.abspath("acestep/third_parts/nano-vllm"))
    print("⚠️ 직접 import에 실패하여 PYTHONPATH를 조정합니다.")

### ⚙️ 3. 실행 설정 및 네트워킹

우측의 **폼(Form)**을 사용하여 접속 및 모델 설정을 완료하세요.

In [ ]:
# @title 🛠️ ACE-Step 구성 (Configuration) { display-mode: "form" }

# @markdown --- 
# @markdown ### 🌐 접속 방식 설정 (Access)
USE_GRADIO_SHARE = True # @param {type:"boolean"}
USE_TAILSCALE = False # @param {type:"boolean"}
TAILSCALE_AUTHKEY = "" # @param {type:"string"}

# @markdown --- 
# @markdown ### 🎵 모델 및 초기화 설정 (Models)
MODEL_TYPE = "acestep-v15-turbo" # @param ["acestep-v15-turbo", "acestep-v15-base"]
INIT_LLM = True # @param {type:"boolean"}

# @markdown ---

import os
import time

# [Tailscale Setup Logic]
if USE_TAILSCALE:
    if not TAILSCALE_AUTHKEY:
        try:
            from google.colab import userdata
            TAILSCALE_AUTHKEY = userdata.get('TAILSCALE_AUTHKEY')
        except: pass

    if TAILSCALE_AUTHKEY:
        print("📥 Tailscale 설치 중...")
        !curl -fsSL https://tailscale.com/install.sh | sh > /dev/null
        
        print("⚙️ Tailscale 데몬 시작 (Userspace Mode)...")
        !pkill tailscaled
        !nohup tailscaled --tun=userspace-networking --socks5-server=localhost:1055 --socket=/tmp/tailscaled.sock > /tmp/tailscaled.log 2>&1 &
        time.sleep(5)
        
        print("🔑 Tailscale 인증 및 연결...")
        !tailscale --socket=/tmp/tailscaled.sock up --authkey=$TAILSCALE_AUTHKEY --hostname=colab-acestep --accept-dns=false
        
        print("📂 포워딩 설정 중...")
        !tailscale --socket=/tmp/tailscaled.sock serve --bg 7860
        
        print("✅ Tailscale 연결 성공!")
        !tailscale --socket=/tmp/tailscaled.sock ip -4
        print("🔗 접속 주소: http://colab-acestep (테일넷 기기용)")
    else:
        print("⚠️ Tailscale Auth Key가 없어 설정을 뛰어넘습니다.")
        USE_TAILSCALE = False

# [Prepare Launch Flags]
share_flag = "--share" if USE_GRADIO_SHARE else ""
llm_flag = "--init_llm true" if INIT_LLM else "--init_llm false"
print("✅ 설정이 완료되었습니다. 아래의 실행 셀을 눌러 ACE-Step을 구동하세요.")

### 🚀 4. ACE-Step 구동 (Launch)

In [ ]:
# @title 🚀 Click to Start ACE-Step
print(f"🔥 ACE-Step Launching...\nModel: {MODEL_TYPE}\nShare: {USE_GRADIO_SHARE}\nTailscale: {USE_TAILSCALE}")

# nano-vllm 인식 보장을 위해 PYTHONPATH를 현재 디렉토리와 third_parts로 강제 지정
launch_env = "MPLBACKEND=Agg PYTHONPATH=.:./acestep/third_parts/nano-vllm"

!{launch_env} uv run acestep {share_flag} --server-name 0.0.0.0 --init_service true --config_path {MODEL_TYPE} {llm_flag}